In [ ]:
from androguard.misc import AnalyzeAPK
import matplotlib.pyplot as plt
import numpy as np
from random import randrange
from IPython.display import clear_output
import random
import json
import glob
import time
import sys
import cv2
import os

In [ ]:
# 200 API method calls we collected by ananlysing 10k APKs.
with open('/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/Properties_count/Benign_API_Calls_Dictionary') as file:
    collected_api_calls = json.load(file)

    
for api_call, count in collected_api_calls.items():
    print(api_call, count)

Ljava/util/regex/MatchResult;->start 718
Landroid/bluetooth/le/BluetoothLeScanner;->startScan 592
Landroid/bluetooth/le/BluetoothLeScanner;->stopScan 569
Landroid/bluetooth/le/ScanCallback;-><init> 536
Landroid/bluetooth/BluetoothAdapter;->getBluetoothLeScanner 534
Landroid/bluetooth/le/ScanSettings$Builder;-><init> 524
Landroid/bluetooth/le/ScanSettings$Builder;->build 524
Landroid/bluetooth/le/ScanResult;->getScanRecord 524
Landroid/bluetooth/le/ScanSettings$Builder;->setScanMode 519
Ljava/lang/Math;->nextAfter 508
Landroid/bluetooth/le/ScanResult;->getRssi 494
Landroid/bluetooth/le/ScanRecord;->getBytes 491
[Lcom/facebook/ads/AdSettings$TestAdType;->clone 453
Landroid/media/session/MediaSession;->getCurrentControllerInfo 444
Ljava/util/LinkedHashSet;->removeAll 438
Ljava/util/concurrent/LinkedBlockingDeque;->clear 435
[Lcom/facebook/ads/VideoAutoplayBehavior;->clone 427
Landroid/media/session/MediaSessionManager$RemoteUserInfo;->getPid 423
Landroid/media/session/MediaSessionManager$

In [ ]:
api_calls = list(collected_api_calls.keys())
print(type(api_calls))

<class 'list'>


In [ ]:
benign_api_calls = ['Ljava/lang/Object;.<init>', 'Ljava/lang/StringBuilder;.toString', 'Ljava/lang/StringBuilder;.append',
                      'Landroid/app/Activity;.<init>', 'Ljava/lang/StringBuilder;.<init>', 'Ljava/lang/String;.equals',
                      'Ljava/util/ArrayList;.<init>', 'Ljava/util/Iterator;.next', 'Ljava/util/Iterator;.hasNext',]

benign_api_calls_pixels = [sum(api_call.encode())%256 for api_call in benign_api_calls]
print(len(benign_api_calls_pixels))
print(benign_api_calls_pixels)

9
[220, 239, 13, 176, 195, 89, 60, 124, 152]


In [ ]:
bening_permissions = ['READ_CALL_LOG', 'WRITE_CALL_LOG', 'NFC', 'SET_ALARM', 'READ_PROFILE', 'READ_USER_DICTIONARY', 
                      'WRITE_USER_DICTIONARY', 'SET_TIME_ZONE', 'WRITE_PROFILE', 'BIND_DEVICE_ADMIN', 'SET_PROCESS_FOREGROUND', 
                      'BIND_REMOTEVIEWS', 'BIND_ACCESSIBILITY_SERVICE', 'WRITE_SOCIAL_STREAM', 'READ_SOCIAL_STREAM', 'ADD_VOICEMAIL', 
                      'BIND_VPN_SERVICE', 'SET_POINTER_SPEED', 'BIND_TEXT_SERVICE', 'USE_CREDENTIALS', 'MANAGE_ACCOUNTS', 
                      'CHANGE_COMPONENT_ENABLED_STATE', 'ACCESS_MOCK_LOCATION', 'AUTHENTICATE_ACCOUNTS', 'CAMERA', 
                      'CHANGE_WIFI_MULTICAST_STATE', 'READ_EXTERNAL_STORAGE', 'READ_CALENDAR', 'FLASHLIGHT', 'READ_SYNC_STATS', 
                      'GET_ACCOUNTS', 'CLEAR_APP_USER_DATA', 'BROADCAST_STICKY',
                     ]

benign_pixels = [sum(perm.encode())%256 for perm in bening_permissions]
print(len(benign_pixels))
print(benign_pixels)

33
[216, 71, 215, 184, 140, 15, 126, 21, 251, 244, 196, 214, 180, 208, 97, 193, 224, 60, 49, 122, 104, 226, 243, 62, 169, 41, 82, 181, 230, 166, 159, 190, 201]


In [ ]:
def OneD2TwoD_resize(image_pixels):

    h = int(len(image_pixels)**0.5)
    w = int(len(image_pixels)/h)
    
    unused_valus = len(image_pixels) - h*w
    inc = 0
    if(unused_valus != 0):
        unused_valus = len(image_pixels) - h*w
        empty_spots = w-unused_valus
        i = unused_valus
        j = unused_valus+empty_spots
        image_pixels += image_pixels[-j:-i]
        inc+=1
    
    image_pixels = np.array(image_pixels)
    twoD_image = image_pixels.reshape(h+inc,w)
    
    twoD_image = twoD_image.astype('float32')

    if(twoD_image.shape[0]>64):
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_LINEAR)
    else:
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_LINEAR)
    return image_resized

In [ ]:
def dex2Image(dx,d):
    #Convert APIs to pixel values
    blue_channel_stuff, API_calls_pixels = API_calls2pixels(dx)
    
    #Convert Opcodes to pixel values
    opcodes_pixls = list(opcodes2pixels(dx))
    
    red_cahnnel = API_calls_pixels + opcodes_pixls  #[API_calls_pixels + opcodes_pixls][0]

    #Convert Opcodes to pixel values
    opcodes_pixls = list(opcodes2pixels(dx))
    #blue_channel_stuff += opcodes_pixls
    
    
    #Extract malicious strings
    blue_channel_stuff += strings2pixels(d)
    
    return red_cahnnel, blue_channel_stuff

def API_calls2pixels(dx):
    imprtnt_API_calls = []
    nrml_API_calls = []
    for api in dx.get_external_classes():
        api_tmp = api.get_vm_class()
        for i in api_tmp.get_methods():
            api_call = str(i).split('(')[0] # ignoring parameters and return type.
            if(api_call in API_calls_dict):
                imprtnt_API_calls.append(API_calls_dict[api_call])
            else:
                nrml_API_calls.append(sum(api_call.encode())%256)
    return imprtnt_API_calls, nrml_API_calls

def opcodes2pixels(dx):
    opcodes = []
    for method in dx.get_methods():
        if method.is_external():
            continue
        m = method.get_method()
        for ins in m.get_instructions():
          #if(ins.get_op_value not in opcodes):
          opcodes.append(ins.get_op_value())
    opcodes = list(set(opcodes))
    return opcodes

def strings2pixels(d):
    string_pixels = []
    all_strings = d[0].get_strings()
    
    for strng in all_strings:
        if(strng in API_calls_dict):
            string_pixels.append(API_calls_dict[strng])
        elif(strng in malicious_permissions):
            string_pixels.append(malicious_permissions[strng])
        elif(strng in malicous_activities):
            string_pixels.append(malicous_activities[strng])
        elif(strng in malicious_services):
            string_pixels.append(malicious_services[strng])
        elif(strng in malicious_recivers):
            string_pixels.append(malicious_recivers[strng])
        elif(strng in malicious_intents):
            string_pixels.append(malicious_intents[strng])
        else:
            pass
    #[string_pixels.append((sum(strng.encode()))%256) for strng in all_strings]
    return string_pixels

In [ ]:
def manifest2Image(a):
    #Permissions
    permissions = a.get_permissions() + a.get_declared_permissions()
    #print(permissions)
    imprtnt_permissions = []
    nrml_permissions = []
    for permission in permissions:
        perm = permission.split('.')[-1]
        # main key word (permission) is oftenly(always) comes at last.
        if(perm in malicious_permissions):
            imprtnt_permissions.append(malicious_permissions[perm])
        elif(perm in decleared_permissions):
            imprtnt_permissions.append(decleared_permissions[perm])
        else:
            nrml_permissions += list(perm.encode())
    
    #Activities
    activities = a.get_activities()
    imprtnt_activities = []
    nrml_activities = []
    for activity in activities:
        act = activity.split('.')[-1]
        if(act in malicous_activities):
            imprtnt_activities.append(malicous_activities[act])
        else:
            nrml_activities += list(act.encode())
            
    #Services
    services = a.get_services()
    imprtnt_services = []
    nrml_services = []
    for service in services:
        srvc = service.split('.')[-1]
        if(srvc in malicious_services):
            imprtnt_services.append(malicious_services[srvc])
        else:
            nrml_services += list(srvc.encode())
            
    #Recivers
    receivers = a.get_receivers()
    imprtnt_receivers = []
    nrml_receivers = []
    for receiver in receivers:
        recevr = receiver.split('.')[-1]
        if(recevr in malicious_recivers):
            imprtnt_receivers.append(recevr)
        else:
            nrml_receivers += list(recevr.encode())
            
    #Providers
    providers = a.get_providers()
    nrml_providers = []
    for provider in providers:
        nrml_providers += list(provider.encode())
        
    #Intents
    imprtnt_intents = []
    nrml_intents = []
    manifest_list = {'permissions':permissions,'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    intents_itemtype = {'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    for itemtype, listt in intents_itemtype.items():
        for item in listt:
            try:
                for intnts in a.get_intent_filters(itemtype, item).values():
                    for intnt in intnts:
                        if(intnt in malicious_intents):
                            imprtnt_intents.append(malicious_intents[intnt])
                        else:
                            nrml_intents += list(intnt.encode())
            except:
                pass
    blue_channel_stuff = imprtnt_permissions + imprtnt_activities + imprtnt_services + imprtnt_intents + imprtnt_receivers
    green_channel = nrml_permissions + nrml_activities + nrml_services + nrml_receivers + nrml_providers + nrml_intents
    green_channel = green_channel
    return green_channel, blue_channel_stuff

In [ ]:
#Appending benign properties to apk file-> to image.
def apk2image_equal(a, d, dx):
    green_channel, blue_channel_stuff1 = manifest2Image(a)
    red_channel, blue_channel_stuff2 = dex2Image(dx, d)
    blue_channel_stuff = blue_channel_stuff1 + blue_channel_stuff2
        
    total_apk_pixels = len(green_channel) + len(red_channel) + len (blue_channel_stuff)
    perturbation_count_red = round((total_apk_pixels*4)/100) #(In totla 12%, 4% on each channel
    perturbation_count_green = round((total_apk_pixels*4)/100)
    #perturbation_pixels = [random.randint(0,256) for i in range(perturbation_count)]
    ##perturbation_pixels = benign_pixels*int(perturbation_count/len(benign_pixels)) + benign_pixels[:int(perturbation_count%len(benign_pixels))]
    
    perturb_pixels_red = benign_api_calls_pixels*int(perturbation_count_red/len(benign_api_calls_pixels)) + benign_api_calls_pixels[:int(perturbation_count_red%len(benign_api_calls_pixels))]
    perturb_pixels_green = benign_permissions_pixels*int(perturbation_count_green/len(benign_permissions_pixels)) + benign_permissions_pixels[:int(perturbation_count_green%len(benign_permissions_pixels))]    
    perturb_pixels_blue = perturb_pixels_green[:len(perturb_pixels_green)//2] + perturb_pixels_red[:len(perturb_pixels_red)//2]
    
    print(f'There are {total_apk_pixels} pixels ectracted from APK.')
    print(f'30 % of total pixels is {perturbation_count_red, perturbation_count_green}')
    print(f'values of 30 % {len(perturb_pixels_red), len(perturb_pixels_green), len(perturb_pixels_blue)}')
    
    green_channel = OneD2TwoD_resize(green_channel + perturb_pixels_green)
    red_channel = OneD2TwoD_resize(red_channel + perturb_pixels_red)
    blue_channel = OneD2TwoD_resize(blue_channel_stuff + perturb_pixels_blue)

    #blue_channel = OneD2TwoD_resize_BlueChannel(blue_channel_stuff)
    
    
    image = cv2.merge((red_channel, green_channel, blue_channel))
    image = image.astype(dtype='uint8')
    return image

## Single Processing

This cell is added.

In [ ]:
#Convert APK to image for Gans
malware_imges_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/Equally_Distributed/_12_Percent_Perturbation/'
malware_APKs_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/11k_100_APKs_AEs_Testing/*'

for apk in glob.glob(malware_APKs_path):
    print('Analysing :', apk[-68:])
    
    #try:
    #Androguard
    a,d,dx = AnalyzeAPK(apk)

    img = apk2image_equal(a,d,dx)

    #Saving to local directory
    cv2.imwrite(os.path.join(malware_imges_path + apk[-68:-4] + '.jpg'), img)
    #print(os.path.join(malware_imges_path + apk[-68:-4] + '.jpg'))
    #except:
    #    pass
    

Analysing : 0A45F03651D5F8EAD4215ED992BCD752EDB466A1EC9F68497DEB9311F83C69F3.apk
There are 5213 pixels ectracted from APK.
30 % of total pixels is (209, 209)
values of 30 % (209, 209, 208)
Analysing : 0B3633D81E79897F4C3ADDF588E6A2242DD68D7619AF4C9F5E927C3E5293D92E.apk
There are 4365 pixels ectracted from APK.
30 % of total pixels is (175, 175)
values of 30 % (175, 175, 174)
Analysing : 0AA81A15738C98198E2AFF9168A1873E1D8DE740D221CFB46E9F7129EF5468D1.apk
There are 2949 pixels ectracted from APK.
30 % of total pixels is (118, 118)
values of 30 % (118, 118, 118)
Analysing : 0A92DAFA9B3D4F8578B61DD5B9A04CAB1BAF3DC2E7137DEA86FCC588D76F2AA0.apk
There are 3922 pixels ectracted from APK.
30 % of total pixels is (157, 157)
values of 30 % (157, 157, 156)
Analysing : 0B603B9A63C5B6F9551CEB7156EF7A023535DD1C87166D8612D38590B6F036DA.apk
There are 1661 pixels ectracted from APK.
30 % of total pixels is (66, 66)
values of 30 % (66, 66, 66)
Analysing : 0B6635B046C0A8F34D6B1112F6238716D743BC645C869EB3

Requested API Level could not be found, using 19 instead


There are 5496 pixels ectracted from APK.
30 % of total pixels is (220, 220)
values of 30 % (220, 220, 220)
Analysing : 0B667568C979E95FDC50920C7A3EF96217A6414E5142064631431369B92699C4.apk
There are 849 pixels ectracted from APK.
30 % of total pixels is (34, 34)
values of 30 % (34, 34, 34)
Analysing : 0A98C166B4978FB7AC02626F36D8C7C7E62DD6744E877473B0639F39F18925AD.apk
There are 2792 pixels ectracted from APK.
30 % of total pixels is (112, 112)
values of 30 % (112, 112, 112)
Analysing : 0A9392A878916AE27D2C49A4DA214D0D50EC34A43469DB27E20AF8070B847E92.apk
There are 467 pixels ectracted from APK.
30 % of total pixels is (19, 19)
values of 30 % (19, 19, 18)
Analysing : 0B70284016AE67EFC68BC410D2A1BA1FECDEBF8DAFFCABAA94BFBCC956D7201E.apk
There are 6032 pixels ectracted from APK.
30 % of total pixels is (241, 241)
values of 30 % (241, 241, 240)
Analysing : 0A17BBD6CC8EED12BAFAC69DAD8424791E6F7A237AD436DFCCF5FAF6501ACE34.apk
There are 5201 pixels ectracted from APK.
30 % of total pixels is (

Requested API Level could not be found, using 19 instead


There are 5492 pixels ectracted from APK.
30 % of total pixels is (220, 220)
values of 30 % (220, 220, 220)
Analysing : 0B0C392BFE932AD524E2CAF4A7F3B3841F8AB896A16C5BC25E667D3DB15DF43D.apk
There are 7195 pixels ectracted from APK.
30 % of total pixels is (288, 288)
values of 30 % (288, 288, 288)
Analysing : 0AEC94353C46C52BCD9082F3477D37F325F19048DC1D594274625810777F9CAA.apk
There are 4316 pixels ectracted from APK.
30 % of total pixels is (173, 173)
values of 30 % (173, 173, 172)
Analysing : 0B90B4381972B7CDD2AE86021638D4A254A0825878E38C57590A843667122C89.apk
There are 6641 pixels ectracted from APK.
30 % of total pixels is (266, 266)
values of 30 % (266, 266, 266)
Analysing : 0B6275CBF121D871225D73356883530CCEA86CE66062423D3876F00CBB0247F0.apk
There are 717 pixels ectracted from APK.
30 % of total pixels is (29, 29)
values of 30 % (29, 29, 28)
Analysing : 0BECD4A4602482ED6EBAF3B1DA00CB1AADBB53F09DCD334433786F957341CD15.apk
There are 1621 pixels ectracted from APK.
30 % of total pixel

There are 1191 pixels ectracted from APK.
30 % of total pixels is (48, 48)
values of 30 % (48, 48, 48)
Analysing : 0A6B6FD483ACC11E157575DF7FCF85F46C21CCF83B5248BC14DFB395CFAFDD40.apk
There are 22777 pixels ectracted from APK.
30 % of total pixels is (911, 911)
values of 30 % (911, 911, 910)
Analysing : 0BEB1E8B6696D5BFDCEC7CEF215A86B64844BA3D5D538E88A8FFD9152989325E.apk
There are 898 pixels ectracted from APK.
30 % of total pixels is (36, 36)
values of 30 % (36, 36, 36)
Analysing : 0B95E1070FC1B946C5B2E8BD8F1FD5BDDD84946DAD3563F38D5307C2AD1B35D1.apk
There are 1980 pixels ectracted from APK.
30 % of total pixels is (79, 79)
values of 30 % (79, 79, 78)
Analysing : 0A8F8E65FC91A869873ACED193966C012864C79ACB5C0BD721EFF4BEAC803A85.apk
There are 1372 pixels ectracted from APK.
30 % of total pixels is (55, 55)
values of 30 % (55, 55, 54)
Analysing : 0A7B8376DAC8C54FB122F9553E9BA8D7DC9DD49EE8BD7283BC896A44A40F4FD1.apk
There are 4877 pixels ectracted from APK.
30 % of total pixels is (195, 195

# Multi Processing

In [ ]:
benign_apks_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Benign_APKs/'
benign_shas_list = os.listdir(benign_apks_path)
benign_shas_list = [sha[:-4] for sha in benign_shas_list]
print(len(benign_shas_list))
print(benign_shas_list[0])

converted_imgs_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/Inter_Linear/Ben_imgs_12K_dplct_signature+extra'
converted_imgs = os.listdir(converted_imgs_path)
converted_imgs_list = [sha[:-4] for sha in converted_imgs]
print(len(converted_imgs_list))
print(converted_imgs_list[0])

apks_list = list(set(benign_shas_list) - set(converted_imgs_list))

print(len(apks_list))

15813
00218EB22D2487A696CC1222E37CD65F4B474B8A88E72580C1D744D011479D9B
12000
00286842F659084C1BBD93979B6068B6C9E285242D2F7CFB990B9F1A2D5BC08E
3813


In [ ]:
imges_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/Inter_Linear/Ben_imgs_12K_16K/'
APKs_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Benign_APKs/'

def apk2image_multi(sha):
    apk = os.path.join(APKs_path + sha + '.apk')
    print('Analysing :', apk[-68:])
    
    try:
        #Androguard
        a,d,dx = AnalyzeAPK(apk)

        img = apk2image(a,d,dx)

        #Saving to local directory
        cv2.imwrite(os.path.join(imges_path + sha + '.jpg'), img)
    except:
        pass
    

In [ ]:
import concurrent.futures
start_time = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(apk2image_multi, apks_list)
    
print(time.time()-start_time)

Analysing : 0049E7302F4E7C91D62D02BD67F087C157306BFF96459173F9B27D7888D9A5E3.apk
Analysing : 00768282B76D029C246C1E09ADACAED91255F03FDCFD9C7E2C9FA6E74E624EBC.apk
Analysing : 001B1BD5A834C13531821D2ED6952DF7D909E186B4CB8F3089250471C487EC32.apk
Analysing : 00115BB2699FE62991BD164C4ECE625FB4F0AEC2C4C8EDF57C4C38BD2FF2E896.apk
Analysing : 001E6019F01871AA3BF5417679E53696B2A633693AB10105FD2BEF388A9BF8AB.apk
Analysing : 00771E261696768FD8FDD69B3C720C8BA393415AF797DA9C150924595C40DDCB.apk
Analysing : 000898CE3EAB41C660D3D821B4A042C14761F94667F20C01CEEC850958F47CC1.apk
Analysing : 0081493652F356BEAD9D91EBA9F9EAFB90CC729BA01309599E3A971A680EAF3B.apk
Analysing : 007E74089DB4D1DA4B266E55268C4879181C2C46873B00D64F7DC9CB5DDDCB5E.apk
Analysing : 006CD0478512B6ECB16D89D44F437ECDEA3FDD13CC1D0FE254D37B07625D1CC2.apk
Analysing : 0045AD9AE874DE1AFEB4A7E0788AF300338EA8F4070E9B821E956311B30B25E5.apk
Analysing : 00183798AE030657F79F3C32DFA9F55D14EB79136AC80CB38862305290BCD4D0.apk
Analysing : 0021FDC0D9563397

Requested API Level could not be found, using 19 instead


Analysing : 007F11665C39F4B07EC212402F9D135C11D1069DFA1DC97AAE947AE270585147.apk
Analysing : 0065C7ECE08162E0D5B1B03700CEC5DD9AB7A416B4437465D5DEEC301559F6D6.apk
Analysing : 00339CB233366BA23F5C9B64ED10FA7A7D682063EA8D0E40F1C76AD9CDBBD159.apk
Analysing : 006FF8B93075BA06C79D65EF4B5840DEE6CBFE57CA89B7816C979E03E85B4E49.apk
Analysing : 0030983AD5A924A272FF4C00B27FA92A19772B6B5701925D870CCA099AB326D0.apk
Analysing : 00564433381C84C7B1C94ADEEFD56E8270C458380FF0715E510F3E3233C5D618.apk
Analysing : 0045F98EE023DE0E05AACD4293B0FF02226B7A970EAC729A21F4866780F12A6D.apk
Analysing : 00769730A38F2158D6191BE8064550CCEE8EB5C5C8AB3113423822D124BA105B.apk
Analysing : 0040081829F54BF910B30C45C236A23C5D6543A191D0BB05FBBCEF7959A1D775.apk
Analysing : 00151188918C90EE9606319C3594CA89E28D513A34BA53A0C362BD79F80EA561.apk
Analysing : 00722BD94A0A06278F566C6BCAB760D5DF168147DD535607A3A62A21A554E65F.apk
Analysing : 0083D179878E90891B47166F5C87FCF4B760B45733A963EB57ABD0F98624E1BA.apk
Analysing : 007A3AA0C08B2C43

Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Analysing : 005E20958BB414C432A5293FF6B674572D8A6EC168EA9F35F7F2B224BD086379.apk
Analysing : 00326DA897B16089F783800ACA491C73A9D0BF2BE402B72C9F8F2480EE7E6BB8.apk
Analysing : 007DAF70DA2BBCF632B7889D6DF5FB637C1DA557A97D146BBC9C735FE4F37D43.apk
Analysing : 006A1387F54E62974D9E62346DAAD96B64FDFDE669F2ECE1ABBC95AF5F9E6DC6.apk
Analysing : 00460A61BCDDE13B246FE461B183C15F03F15BEFB2BC8019B235048225E84087.apk
Analysing : 00374C042291870405C4B597182D616F9A284F37877ECE28454E665C8846BE5F.apk
Analysing : 0038050B537DA03E0E58CD09A897192C1F712321FEBA3FF705EBB2220A2C806B.apk
Analysing : 004EB1658C5E65EA95447E715A523B6CEEF50863A2177F5B58FB6E8EE2DA7F18.apk
Analysing : 004EAFA8F007BAEE78F6F8B832D2A9C0A10F128BA7AA54DA8C12E8EF80651657.apk
Analysing : 0030DFDB3983B57A9C74D4409994B89E70906BE79831CA920428E4D4D00F6BF0.apk
Analysing : 0043FBE3AC8C1E5D0D40F751D9553C48EBC327970D46E3133F15A46F42BCF632.apk
Analysing : 00424461450B60698AA9AFF2610B6059678EDD3017B79D62D0258CD9E25C4C53.apk
Analysing : 008AC3D9283A15F0

...

Analysing : 0020F82BE55E13D987A5C2725A4B9C2ED84977BFC3E5EE695F0801FFD75146F8.apk
Analysing : 000EC4AEA9729184C14FBB17CBD82C1B615BF12FB97C8B96ADE267AA559CA1A9.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Analysing : 00031BA0F95F2312BB64E34E6D56334880DB89EE5D68A737F03E2C7FC43DED93.apk
Analysing : 004389CBC16A10717EC1850380DDA6140677C076D21A61582CA1C31933C0F169.apk
Analysing : 008269D897F52E3D19DBD61109F06B5ADE20FC37A869E8D188A2B0B03572C8B2.apk
Analysing : 0039CE83CDBC56A0198CFB77772DDC83813D88E5A6424DAC894FE6F1A4432673.apk
Analysing : 005F44BE089D695567696768A791DAE12EF9B6F2AB207028D9EEA595E0E3C7D3.apk


Requested API Level could not be found, using 10 instead


Analysing : 008883296C2EF7A239E558CA238C625CDDE9D9F47902526332B60DB21BF931B2.apk
Analysing : 0068C54807D507B6247A495DACA52FCE1E58D7C6D937E4E080818F456FD5D824.apk
Analysing : 0081E5B97EC20968A2BE8AF7E86F6B67B46CE9E3966127A3881F2303746DB742.apk
Analysing : 006A0BF86BD07AA7C67EEC5590803E62D5D1504DA7187243406A1DC878304D16.apk
Analysing : 0080B760132152141CB1F4B3440030B94AC085E4017A92B5C7AE7C8F406953DA.apk
Analysing : 0047BD598C8733439CB75384301EC229551DC16A2F3D9FAC86CB4360FBC1B9C1.apk
Analysing : 004CE9BA2F6AFF7896E1B15D69EAC215AC909DE7C39070941679D21236270BBE.apk
Analysing : 004D008BAC52474AAF402505EB1D5162ECED68C9DBBF73BF43948FE2BA2C5A89.apk
Analysing : 00245530C3043F471A4C33B826304A6FCFE4D36A59077458F786832189190A85.apk
Analysing : 00518AC787A46ABF66D67F02D82E72DB8EDCF5790611C0B1653F07D1EC189465.apk
Analysing : 0025ABA273BD3829EAE366CB213F8FDFA762BE24D98A546A8BE5008BF75653E6.apk
Analysing : 0033B7B9B15A9A70CC4272DDB38A41E7856FC72A45E3CDE925F647E82EC9523D.apk
Analysing : 006F229ABA3B4158

# Important API Calls

In [ ]:
# Risky API_calls and methods
API_calls_dict = {
    'Ljavax/sql/ConnectionEvent;-><init>':250,
    'Ljava/nio/channels/WritableByteChannel;->close':240,
    'Landroid/service/carrier/CarrierService;->stopSelf':230,
    'Landroid/opengl/Matrix;->getClass':220,
    'Landroid/view/ViewStructure;->setCheckable':210,
    'Landroid/text/method/BaseKeyListener;->getInputType':200,
    'Landroid/provider/MediaStore$Images$Media;->wait':190,
    'Ljava/lang/Runtime;->exec':180,
    'Ljava/lang/System;->loadLibrary':180,
    'Landroid/widget/AdapterView;->refreshDrawableState':170,
    'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':170,

    'Ljava/io/BufferedOutputStream;-><init>':160,
    'Ljava/io/FileOutputStream;-><init>':160,

    'Landroid/app/PendingIntent;->send':150,
    'Landroid/app/AlarmManager;->Set':150,
    'Landroid/app/NativeActivity;->getVolumeControlStream':140,
    'Landroid/app/AcitivityManager;->killBackgroudProcess':140,
    
    'Landroid/content/pm/PacakageManager;->removePackageFromPrefe':130,
    'Landroid/content/pm/PacakageManager;->getInastallerPackageName':130,
    'Landroid/content/pm/PacakageManager;->getInstalledPackages':130,
    'Landroid/content/pm/PacakageManager;->getInstalledApplications':130,
    'Landroid/content/Intent;->startActivity':120,
    'Landroid/content/Intent;->getAction':120,
    'Landroid/content/Intent;->setDataAndType':120,
    'Landroid/content/ContentResolver;->delete':115,
    'Landroid/content/ContentResolver;->update':115,
    'Landroid/content/ContentResolver;->insert':115,
    'Landroid/content/Context;->getPackageManager':115,
    'Landroid/content/BroadcastReceiver;->abortBroadcast':115,

    'Landroid/database/sqlote/SQLiteDatabase;->execSQL':110,

    'Landroid/media/MediaRecorder;->MediaRecorder':100,
    'Ljava/net/HttpURLConnection;->connect':95,
    'Ljava/net/URLConnection-;>connect':95,
    'Lorg/apache/http/impl/client;->DefaultHttpClient':90,

    'Ljavax/crypto/Cipher;->getInstance':85,
    'Ljavax/crypto/Cipher;->Init':85,
    'Ljavax/crypto/Cipher;->doFinal':85,

    'Landroid/telephony/TelephonyManager;->getDataActivity':80,
    'Landroid/telephony/TelephonyManager;->getCallState':80,
    'Landroid/telephony/TelephonyManager;->getLine1Number':80,
    'Landroid/telephony/TelephonyManager;->getCellLocation':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/TelephonyManager;->getDeviceId':80,
    'Landroid/telephony/TelephonyManager;->getNetworkType':80,
    'Landroid/telephony/TelephonyManager;->getSimOperator':80,
    'Landroid/telephony/TelephonyManager;->getSimSerialNumber':80,
    'Landroid/telephony/TelephonyManager;->getSimState':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/SmsManager;->sendTextMessage':75,
    'Landroid/telephony/SmsManager;->sendMultipartTextMessage':75,
    'Landroid/telephony/SmsManager;->sendDataMessage':75,
    'Landroid/telephony/SmsMessage;->getServiceCenterAddress':75,
    'Landroid/telephony/gsm/SmsManager;->sendMultipartTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendDataMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayOriginatingAddress':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayMessageBody':70,
    'Landroid/telephony/PhoneStateListener;->onCallStateChanged':65,

    'Ldalvik/system/DexClassLoader;->loadClass':60,
    'Ldalvik/system/PathClassLoader;->loadClass':60,
    'Landroid/net/wifi/WifiInfo;->getSupplicantState':50,
    'Landroid/net/wifi/WifiManager;->getConnectionInfo':40,
    'Landroid/ContentResolver;->query':30, 
    'Landroid/location/LocationManager;->getLastKnownLocation':20,
}

## Collected following Refrences

Android Malware Identification Through Visual Exploration of Disassembly Files. 2017
International Journal of Network Security, Vol.20, No.6, PP.1061-1073, Nov. 2018 (DOI: 10.6633/IJNS.201811 20(6).07) 1061

In [ ]:
# Risky API_calls and methods
malicious_apis_1 = {'Ljava/lang/Runtime->exec': 220, 'Landroid/content/Intent->startActivity': 210,
                 'Landroid/app/PendingIntent->send': 200, 'Landroid/app/AlarmManager->Set': 200,
                 'Landroid/content/pm/PacakageManager->removePackageFromPrefe': 190,
                 'Landroid/database/sqlote/SQLiteDatabase->execSQL': 180,
                 'Landroid/content/ContentResolver->delete': 170,
                 'Landroid/app/AcitivityManager->killBackgroudProcess': 160,
                 'Landroid/media/MediaRecorder->MediaRecorder': 150, 'Ljava/net/HttpURLConnection->connect': 140,
                 'Ljava/net/URLConnection->connect': 140, 'Lorg/apache/http/impl/client->DefaultHttpClient': 140,
                 'Landroid/content/BroadcastReceiver->abortBroadcast': 63,
                 'Landroid/telephony/PhoneStateListener->onCallStateChanged': 130,
                 'Landroid/content/Intent->getAction': 120, 'Ljavax/crypto/Cipher->getInstance': 110,
                 'Ljavax/crypto/Cipher->Init': 110, 'Ljavax/crypto/Cipher->doFinal': 110,
                 'Landroid/telephony/TelephonyManager->getLine1Number': 100,
                 'Landroid/content/pm/PacakageManager->getInastallerPackageName': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledPackages': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledApplications': 90,
                 'Landroid/location/LocationManager->getLastKnownLocation': 80,
                 'Landroid/telephony/TelephonyManager->getCellLocation': 80,
                 'Landroid/telephony/TelephonyManager->getSubscriberId': 71,
                 'Landroid/telephony/TelephonyManager->getDeviceId': 70,
                 'Landroid/telephony/SmsManager->sendTextMessage': 64,
                 'Landroid/telephony/gsm/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/gsm/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->sendTextMessage': 61,
                 'Landroid/telephony/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->getDisplayOriginatingAddress': 60,
                 'Landroid/telephony/gsm/SmsManager->getDisplayMessageBody': 60,
                 'Ldalvik/system/DexClassLoader->loadClass': 50, 'Ldalvik/system/PathClassLoader->loadClass': 50,
                 'Landroid/content/ContentResolver->update': 40, 'Landroid/content/ContentResolver->insert': 30,
                 'Landroid/ContentResolver->query': 20, 'Landroid/content/Intent->setDataAndType': 10
                }

Android Malware Detection based on Useful API Calls and Machine Learning 2018 IEEE First International Conference on Artificial Intelligence and Knowledge Engineering (AIKE)

In [ ]:
# you have to cehck whether Aandrogurad call method via '.' or '->'
malicious_apis_2 = {'Ljavax/sql/ConnectionEvent;-><init>':255, 'Ljava/nio/channels/WritableByteChannel;->close':250,
 'Landroid/service/carrier/CarrierService;->stopSelf':245 ,
 'Landroid/app/NativeActivity;->getVolumeControlStream':240, 
 'Landroid/widget/AdapterView;->refreshDrawableState':235, 'Landroid/opengl/Matrix;->getClass':230, 
 'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':225, 
 'Landroid/view/ViewStructure;->setCheckable':220, 'Landroid/text/method/BaseKeyListener;->getInputType':215, 
 'Landroid/provider/MediaStore$Images$Media;->wait':210
}

benign_api_1 = {'Ljava/lang/Object;.<init>':0, 'Ljava/lang/StringBuilder;.toString':5, 'Ljava/lang/StringBuilder;.append':10,
 'Landroid/app/Activity;.<init>':15, 'Ljava/lang/StringBuilder;.<init>':20, 'Ljava/lang/String;.equals':25,
 'Ljava/util/ArrayList;.<init>':30, 'Ljava/util/Iterator;.next':35,
 'Ljava/util/Iterator;.hasNext':40
}

Feature extraction from Android application packages and its usage in machine learning for malware classification. Computer Science Institute of Charles University 2017

In [ ]:
malicious_apis_3 = {'Ljava.io.BufferedOutputStream;-><init>':205, 'Ljava.io.FileOutputStream;-><init>':200,
 'Ljava.lang.System;->loadLibrary':195, 
 'Landroid.content.Context;->getPackageManager':190
}

Mining API Calls and Permissions for Android Malware Detection (2014)
The LNM Institute of Information Technology, Jaipur, India 

In [ ]:
malicious_apis_4 = {'Landroid/telephony/TelephonyManager;->getCallState':185,
 'Landroid/telephony/TelephonyManager;->getDataActivity':180,
 'Landroid/telephony/TelephonyManager;->getDeviceId':175,
 'Landroid/telephony/TelephonyManager;->getLine1number':170,
 'Landroid/telephony/TelephonyManager;->getNetworkType':165,
 'Landroid/telephony/TelephonyManager;->getSimOperator':160,
 'Landroid/telephony/TelephonyManager;->getSimSerialNumber':155,
 'Landroid/telephony/TelephonyManager;->getSimState':150,
 'Landroid/telephony/TelephonyManager;->getSubscriberId':145
}

## Important Permissions

Android Malware Detection Using Fine-Grained Features 
Hindawi Scientific Programming Volume 2020

In [ ]:
#Activity
malicous_activities = {
    'ACCESS_COARSE_LOCATION':250,
    'ACCESS_FINE_LOCATION':240,
    'READ_SMS':230,
    'SEND_SMS':220,
    'READ_PHONE_STATE':210,
    'READ_CONTACTS':200,
    'READ_HISTORY_BOOKMARKS':190,
    'MOUNT_UNMOUNT_FILESYSTEMS':180,
    'RECEIVE_SMS':170
}



In [ ]:
#Services
malicious_services = {
    'SEND_SMS':160,
    'READ_SMS':150,
    'ACCESS_COARSE_LOCATION':140,
    'ACCESS_FINE_LOCATION':130,
    'READ_PHONE_STATE':120
}


In [ ]:
#BroadCast Reecvier
malicious_recivers = {
    'SEND_SMS':110,
    'READ_SMS':100,
    'ACCESS_COARSE_LOCATION':90,
    'ACCESS_FINE_LOCATION':80,
    'READ_PHONE_STATE':70
}


In [ ]:
#Intents
malicious_intents = {
    'BOOT_COMPLETED':60,
    'SMS_RECEIVED':50,
    'BATTERY_CHANGE_ACTION':40,
    'MOUNT_UNMOUNT_FILESYSTEMS':30
}


In [ ]:
#Not used but decleared
decleared_permissions={
    'CAMERA':250,
    'RECORD_AUDIO':230,
    'READ_CONTACTS':210,
    'READ_CALL_LOG':200,
    'CALL_PHONE':190,
    'WRITE_CALL_LOG':180,
    'CHANGE_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'READ_CALENDAR':140,
    'ACCESS_WIFI_STATE':130,
    'ACCESS_COARSE_LOCATION':120,
    'ACCESS_FINE_LOCATION':110,
    'READ_PHONE_STATE':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'ACCESS_NETWORK_STATE':60,
    'WRITE_CALENDAR':50,
    'SEND_SMS':40,
    'READ_SMS':30,
    }

In [ ]:
#Permissions
malicious_permissions = {
    'CAMERA':250,
    'RECORD_AUDIO':240,
    'READ_CONTACTS':230,
    'READ_CALL_LOG':220,
    'CALL_PHONE':210,
    'WRITE_CALL_LOG':200,
    'CHANGE_WIFI_STATE':190,
    'READ_CALENDAR':180,
    'ACCESS_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'WAKE_LOCK':150,
    'GET_TASKS':130,
    'VIBRATE':120,
    'PROCESS_OUTGOING_CALLS':110,
    'SYSTEM_ALERT_WINDOWS':100,
    'WRITE_SETTING':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'WRITE_CALENDAR':60,
    'ACCESS_NETWORK_STATE':50,
    'READ_HISTORY_BOOKMARKS':40
}

In [ ]:
Analysing : 0A45F03651D5F8EAD4215ED992BCD752EDB466A1EC9F68497DEB9311F83C69F3.apk
There are 5213 pixels ectracted from APK.
30 % of total pixels is (224, 224)
values of 30 % (224, 224, 224)
Analysing : 0B3633D81E79897F4C3ADDF588E6A2242DD68D7619AF4C9F5E927C3E5293D92E.apk
There are 4365 pixels ectracted from APK.
30 % of total pixels is (188, 188)
values of 30 % (188, 188, 188)
Analysing : 0AA81A15738C98198E2AFF9168A1873E1D8DE740D221CFB46E9F7129EF5468D1.apk
There are 2949 pixels ectracted from APK.
30 % of total pixels is (127, 127)
values of 30 % (127, 127, 126)
Analysing : 0A92DAFA9B3D4F8578B61DD5B9A04CAB1BAF3DC2E7137DEA86FCC588D76F2AA0.apk
There are 3922 pixels ectracted from APK.
30 % of total pixels is (169, 169)
values of 30 % (169, 169, 168)
Analysing : 0B603B9A63C5B6F9551CEB7156EF7A023535DD1C87166D8612D38590B6F036DA.apk
There are 1661 pixels ectracted from APK.
30 % of total pixels is (71, 71)
values of 30 % (71, 71, 70)
Analysing : 0B6635B046C0A8F34D6B1112F6238716D743BC645C869EB3E36CCF7F5E7B94DD.apk
There are 2721 pixels ectracted from APK.
30 % of total pixels is (117, 117)
values of 30 % (117, 117, 116)
Analysing : 0BBE34759E3D0F5317DEFBF52C0CEC8A317F51935878C55C29DD6D4EFCEE6232.apk
There are 5523 pixels ectracted from APK.
30 % of total pixels is (237, 237)
values of 30 % (237, 237, 236)
Analysing : 0BD48FAFBD2A303E91E9603CDF03E9A1B03ECDFD2BA19A931951E5036731FD28.apk